#### Import the library 
Note: An error might popup after this command has exectuted. If it does read it careful before ignoring. 

In [1]:
import psycopg2

### Create a connection to the database

In [2]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

### Next use that connect to get a cursor that we will use to execute queries.

In [3]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)

### Then we create a function to execute select queries and exibing the result. We'll catch the error if exists.

In [4]:
def execute_select(cur, query):
    try: 
        cur.execute(query)
        
    except psycopg2.Error as e: 
        print("Error: select *")
        print (e)

    row = cur.fetchone()
    while row:
       print(row)
       row = cur.fetchone()

### Here we have the representation of our Star Schema with one fact table and four dimension tables coming from it.

`Table Name: songplays
column: songplay_id
column: start_time
column: user_id
column: level
column: song_id
column: artist_id
column: session_id
column: location
column: user_agent`

`Table Name: users
column: user_id
column: first_name
column: last_name
column: gender
column: level`

`Table Name: songs
column: song_id
column: title
column: artist_id
column: year
column: duration`

`Table Name: artists
column: artist_id
column: name
column: location
column: latitude
column: longitude`

`Table Name: time
column: start_time
column: hour
column: day
column: week
column: month
column: year
column: weekday`

<img src="images/StarSchema.png" width="750">

## Now we will run some queries on this database to analyze the Sparkify.

#### The first information we want about the Sparkify is understand the main artists and songs.

#### Top 10 artists
The problem here is the fact of only one row has a song_id associated.

In [5]:
execute_select(cur, 'SELECT songplays.songplay_id, songplays.song_id, songs.title FROM songplays INNER JOIN songs ON songplays.song_id = songs.song_id')

(3225, 'SOZCTXZ12AB0182364', 'Setanta matins')


### Top 10 songs
The problem here is the same that we find with artists

In [6]:
execute_select(cur, 'SELECT songplays.songplay_id, songplays.artist_id, artists.name FROM songplays INNER JOIN artists ON songplays.artist_id = artists.artist_id')

(3225, 'AR5KOSW1187FB35FF4', 'Elena')


### As this two columns won't give us so many information, we can focus on others importants things

## Top 10 locations where the Sparkify is used

In [7]:
execute_select(cur, 'SELECT COUNT(songplay_id), location FROM songplays GROUP BY location ORDER BY COUNT(songplay_id) DESC LIMIT 10')

(691, 'San Francisco-Oakland-Hayward, CA')
(665, 'Portland-South Portland, ME')
(557, 'Lansing-East Lansing, MI')
(475, 'Chicago-Naperville-Elgin, IL-IN-WI')
(456, 'Atlanta-Sandy Springs-Roswell, GA')
(397, 'Waterloo-Cedar Falls, IA')
(321, 'Lake Havasu City-Kingman, AZ')
(307, 'Tampa-St. Petersburg-Clearwater, FL')
(292, 'San Jose-Sunnyvale-Santa Clara, CA')
(270, 'Sacramento--Roseville--Arden-Arcade, CA')


### We can see how is the Sparkify's users by gender. It's possible to see a little more women using the program.

In [8]:
execute_select(cur,'SELECT COUNT(user_id), gender FROM users GROUP BY gender')

(55, 'F')
(41, 'M')


### But even with a similar number of users, women use much more than men

In [9]:
execute_select(cur, 'SELECT COUNT(songplays.songplay_id), users.gender FROM songplays INNER JOIN users ON songplays.user_id = users.user_id GROUP BY users.gender')

(1933, 'M')
(4887, 'F')


### The Top 10 users confirm that women use more Sparkify than man when we see seven women in top 10

In [10]:
execute_select(cur, '''
SELECT COUNT(songplays.songplay_id), users.last_name, users.first_name, users.gender
FROM songplays INNER JOIN users ON songplays.user_id = users.user_id 
GROUP BY users.last_name, users.first_name, users.gender
ORDER BY COUNT(songplays.songplay_id) DESC, users.last_name, users.first_name 
LIMIT 10
''')

(689, 'Cuevas', 'Chloe', 'F')
(665, 'Levine', 'Tegan', 'F')
(557, 'Harrell', 'Kate', 'F')
(463, 'Koch', 'Lily', 'F')
(397, 'Kirby', 'Aleena', 'F')
(346, 'Lynch', 'Jacqueline', 'F')
(321, 'Griffin', 'Layla', 'F')
(289, 'Klein', 'Jacob', 'M')
(270, 'Rodriguez', 'Mohammad', 'M')
(248, 'Jones', 'Matthew', 'M')


### Other important information is the number of paid and free user using the Sparkify. We can see much more free users

In [11]:
execute_select(cur,'SELECT COUNT(user_id), level FROM users GROUP BY level')

(74, 'free')
(22, 'paid')


### But the paid user are responsable for the most part of the song plays in Sparkify

In [12]:
execute_select(cur,'SELECT COUNT(songplays.songplay_id), users.level FROM songplays INNER JOIN users ON songplays.user_id = users.user_id GROUP BY users.level')

(1115, 'free')
(5705, 'paid')


### This information is confirmed for Top 10 users because the users who most use the Sparkify are all paid accounts

In [13]:
execute_select(cur, '''
SELECT COUNT(songplays.songplay_id), users.last_name, users.first_name, users.level
FROM songplays INNER JOIN users ON songplays.user_id = users.user_id 
GROUP BY users.last_name, users.first_name, users.level
ORDER BY COUNT(songplays.songplay_id) DESC, users.last_name, users.first_name 
LIMIT 10
''')

(689, 'Cuevas', 'Chloe', 'paid')
(665, 'Levine', 'Tegan', 'paid')
(557, 'Harrell', 'Kate', 'paid')
(463, 'Koch', 'Lily', 'paid')
(397, 'Kirby', 'Aleena', 'paid')
(346, 'Lynch', 'Jacqueline', 'paid')
(321, 'Griffin', 'Layla', 'paid')
(289, 'Klein', 'Jacob', 'paid')
(270, 'Rodriguez', 'Mohammad', 'paid')
(248, 'Jones', 'Matthew', 'paid')


### So, from this database we could take some conclusions and understand better how the Sparkify is used. 